In [1]:
import tensorflow as tf
import numpy as np
import datetime, os

In [2]:
from scipy import io as sio

In [3]:
with tf.Session() as sess:
  devices = sess.list_devices()

In [4]:
devices

[_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456, 6817330511422053373),
 _DeviceAttributes(/job:localhost/replica:0/task:0/device:GPU:0, GPU, 9205221950, 11710985869349011853)]

In [5]:
emnist = sio.loadmat("emnist-letters.mat")
x_train = emnist["dataset"]['train'][0,0]['images'][0,0]
#x_train = x_train.astype(np.float32)

y_train = emnist["dataset"]['train'][0,0]['labels'][0,0]

x_test = emnist["dataset"]['test'][0,0]['images'][0,0]
#x_test = x_test.astype(np.float32)

y_test = emnist["dataset"]['test'][0,0]['labels'][0,0]

In [6]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

In [7]:
x_train, x_test = x_train / 255.0, x_test / 255.0

In [8]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
lb.fit([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26])
#lb.classes_array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26])
y_train = lb.transform(y_train)
y_test = lb.transform(y_test)

In [9]:
import matplotlib.pyplot as plt

img = x_train[15]
plt.imshow(img[0], cmap='gray')

In [10]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), input_shape=(28, 28, 1), name="Input-Conv-1", padding='same'),
#    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(128, (3,3), name= "Conv-2", padding='same'),
#    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(256, (3,3), name="Conv-3", padding='same'),
#    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation="relu", name="Dense"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(32, activation="relu", name="Denser"),
    tf.keras.layers.Dense(26, activation="softmax", name="Output-Dense")
])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [11]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input-Conv-1 (Conv2D)        (None, 28, 28, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 64)        0         
_________________________________________________________________
Conv-2 (Conv2D)              (None, 14, 14, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 128)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 128)         0         
_________________________________________________________________
Conv-3 (Conv2D)              (None, 7, 7, 256)         295168    
__________

In [13]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [14]:
model.fit(x_train, y_train, epochs=20, validation_split=0.25)

Train on 93600 samples, validate on 31200 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
93600/93600 [==============================] - 19s 204us/sample - loss: 0.5483 - acc: 0.8338 - val_loss: 0.3563 - val_acc: 0.8889
Epoch 2/20
93600/93600 [==============================] - 17s 182us/sample - loss: 0.2906 - acc: 0.9046 - val_loss: 0.2577 - val_acc: 0.9137
Epoch 3/20
93600/93600 [==============================] - 17s 181us/sample - loss: 0.2506 - acc: 0.9159 - val_loss: 0.2178 - val_acc: 0.9292
Epoch 4/20
93600/93600 [==============================] - 17s 182us/sample - loss: 0.2222 - acc: 0.9254 - val_loss: 0.2393 - val_acc: 0.9232
Epoch 5/20
93600/93600 [==============================] - 17s 180us/sample - loss: 0.2059 - acc: 0.9295 - val_loss: 0.1901 - val_acc: 0.9364
Epoch 6/20
93600/93600 [==============================] - 17s 179us/sample - loss: 0.1982 - acc: 0.9318 - val_loss: 0.1955 - val_acc: 0.9361
Epoch 7/20
93600/93600 [==============================] 

In [15]:
model.evaluate(x_test, y_test)

20800/20800 [==============================] - 1s 49us/sample - loss: 0.2041 - acc: 0.9410


[0.20411638752636813, 0.94100964]

In [16]:
tf.keras.models.save_model(model, "./94-1_accuracy.mdl")

In [ ]:
%tensorboard --logdir logs

In [ ]:
for i in range(0, y_train.shape[0]):
    y_train[i] = y_train[i] -1

In [ ]:
for i in range(0, y_test.shape[0]):
    y_test[i] = y_test[i] +1